In [2]:
import random
import xml.etree.ElementTree as ET

import pandas
from plotnine import *
import requests
from tqdm import tqdm_notebook

%load_ext rpy2.ipython

In [3]:
tree = ET.parse('../data/tree.xml')
root = tree.getroot()

In [4]:
studies = root.findall('EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE/STUDY')

study_accessions = [study.get('accession') for study in studies]
# Remove duplicate accessions
study_accessions = list(set(study_accessions))
print(study_accessions[:5])

['SRP080879', 'SRP011998', 'SRP034698', 'SRP092544', 'SRP109264']


In [5]:
%%R
library(recount)

# Download the metadata for all samples in recount, and write it to a file
metadata <- all_metadata()
write.table(metadata, '~/Desktop/brd-net/data/recount_metadata.csv', sep='\t', row.names=FALSE)

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: SummarizedExperiment

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: GenomicRanges

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: stats4

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: BiocGenerics

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: parallel

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinter

In [6]:
# Load the metadata from calling recount's all_metadata function
recount2_metadata = pandas.read_csv('../data/recount_metadata.csv', sep='\t')

# Create the set of all project accessions present in recount2
recount2_study_accessions = set(recount2_metadata['project'])
print(len(recount2_study_accessions))

3219


In [7]:
# Keep only the accessions that can be downloaded from recount2
study_accessions = [accession for accession in study_accessions if accession in recount2_study_accessions]
print(len(study_accessions))

239


In [8]:
# Download information about a single study to show the data format returned from SRA
data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(study_accessions[0]))
print(data.json())

{'sampleCount': 8, 'terms': [{'sampleCount': 8, 'dterm': {'name': 'Merkel cell carcinoma', 'ids': ['DOID:3965']}}, {'sampleCount': 8, 'dterm': {'name': 'basal cell carcinoma', 'ids': ['DOID:2513', 'EFO:0004193']}}, {'sampleCount': 8, 'dterm': {'name': 'carcinoma', 'ids': ['EFO:0000313']}}, {'sampleCount': 8, 'dterm': {'name': 'cell line', 'ids': ['EFO:0000322']}}, {'sampleCount': 8, 'dterm': {'name': 'cultured cell', 'ids': ['CL:0000010']}}, {'sampleCount': 8, 'dterm': {'name': 'lymph node', 'ids': ['UBERON:0000029']}}, {'sampleCount': 8, 'dterm': {'name': 'melanoma', 'ids': ['DOID:1909', 'EFO:0000756']}}, {'sampleCount': 8, 'dterm': {'name': 'semicircular canal', 'ids': ['UBERON:0001840']}}, {'sampleCount': 8, 'dterm': {'name': 'skin', 'ids': ['EFO:0000962']}}, {'sampleCount': 8, 'dterm': {'name': 'skin neoplasm', 'ids': ['EFO:0004198']}}, {'sampleCount': 8, 'dterm': {'name': 'squamous cell carcinoma', 'ids': ['EFO:0000707', 'DOID:1749']}}, {'sampleCount': 8, 'dterm': {'name': 'zone o

In [9]:
id_to_studies = {}
id_to_samples = {}
id_to_name = {}

total_samples = 0

# Find the disease ontology information associated with all the studies in both recount2 and 
# the output of find_studies.py
for accession in tqdm_notebook(study_accessions):
    data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(accession)).json()
    
    total_samples += data['sampleCount']
    
    for term in data['terms']:
        term_name = term['dterm']['name']
        term_ids = term['dterm']['ids']
        
        # Iterate over all ids in term_ids, and return the disease ontology id.
        # If there is no doid for the term, continue to the next term.
        disease_id = next((id_ for id_ in term_ids if 'DOID' in id_), None)
        if disease_id is None:
            continue
        
        id_to_studies[disease_id] = id_to_studies.setdefault(disease_id, 0) + 1
        id_to_samples[disease_id] = id_to_samples.setdefault(disease_id, 0) + term['sampleCount']
        id_to_name[disease_id] = term_name

In [10]:
disease_dict = {}
for id_ in id_to_studies:
    disease_dict[id_] = [id_to_studies[id_], id_to_samples[id_], id_to_name[id_]]

disease_df = pandas.DataFrame.from_dict(disease_dict, orient='index', columns=['num_studies', 'num_samples', 'name'])

all_doids = set(id_to_studies.keys())

disease_df.sort_values('num_samples').head()

,num_studies,num_samples,name
DOID:11722,1,6,myotonic dystrophy type 1
DOID:1974,1,6,adenosarcoma
DOID:12849,1,6,autistic disorder
DOID:3965,1,8,Merkel cell carcinoma
DOID:397,1,8,restrictive cardiomyopathy


In [11]:
DOID_to_descendants = {}
def get_descendants(doid):
    '''This function calls the Disease Ontology API to find the ids of all descendants of a given doid'''
    if doid in DOID_to_descendants:
        return DOID_to_descendants[doid]
    
    descendants = []
    data = requests.get('http://www.disease-ontology.org/api/metadata/{}'.format(doid)).json()
    
    if 'children' not in data:
        return []
    
    for child in data['children']:
        child_doid = child[1]
        descendants.append(child_doid)
        
        # recursively traverse the ontology
        child_descendants = get_descendants(child_doid)
        descendants.extend(child_descendants)
        
    DOID_to_descendants[doid] = descendants
    return descendants

In [12]:
# Iterate through all terms found in the data, and only add them to a list if 
# none of their descendants are in the study

leaf_ids = []

# Because this function involves recursively traversing a tree, the estimated time
# given by tqdm will be way off. That said, it takes awhile to run (~ 20 minutes)

for doid in tqdm_notebook(id_to_name):
    descendants = get_descendants(doid)
    descendant_in_study = False
    for descendant in descendants:
        if descendant in all_doids:
            descendant_in_study = True
            break
    
    if not descendant_in_study:
        leaf_ids.append(doid)

# These aren't necessarily leaves of the ontology, but they are don't have any descendants
# in this dataset, so we'll call them leaves
print(len(leaf_ids))


102


In [13]:
disease_df.loc[leaf_ids].head()

,num_studies,num_samples,name
DOID:3965,1,8,Merkel cell carcinoma
DOID:2513,2,29,basal cell carcinoma
DOID:1909,8,239,melanoma
DOID:1749,2,46,squamous cell carcinoma
DOID:3181,1,49,oligodendroglioma


In [14]:
print(total_samples)
print(disease_df.loc[leaf_ids]['num_samples'].sum())

13437
8877


In [15]:
# Select studies at random from the list of all samples
# If the study has one of the leaf ids, add it to the list
# Stop once enough samples are included (~1000 valid samples)

# In a script, you would put the random seed at the top. Since each cell can be
# rerun in a notebook, manually setting the seed within each cell with a random
# call ensures that you get the same results regardless of the order in which
# the cells are run
random.seed(42)

leaf_set = set(leaf_ids)

samples_drawn = 0
plier_sample_accessions = []

# Select samples in a random order
indices = list(range(len(study_accessions)))
random.shuffle(indices)

for index in tqdm_notebook(indices):
    study = study_accessions[index]
    data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(study)).json()

    for term in data['terms']:
        term_ids = term['dterm']['ids']

        use_sample = False
        for id_ in term_ids:
            if id_ in leaf_set:
                plier_sample_accessions.append(study)
                
                samples_drawn += data['sampleCount']
                use_sample = True
                break
        
        if use_sample:
            break
            
    # Some of the samples will turn out to be invalid, so we use 3000 here instead of
    # a proportion of the total number of samples
    if samples_drawn > 3000:
        break
        
print(plier_sample_accessions)

['SRP033566', 'SRP045500', 'SRP042184', 'SRP056792', 'SRP049820', 'SRP003611', 'SRP057118', 'SRP063661', 'SRP012656', 'SRP011422', 'SRP037982', 'SRP056041', 'SRP033095', 'SRP002272', 'SRP052896', 'SRP007947', 'SRP009123', 'ERP002414', 'SRP017789', 'SRP061888', 'SRP028554', 'SRP063838', 'SRP061240', 'SRP017413', 'SRP056612', 'SRP051765', 'SRP029434', 'SRP031459', 'SRP017262', 'SRP049097', 'SRP002628', 'SRP062966', 'SRP035988', 'SRP015640', 'SRP034698', 'SRP048801', 'SRP041538', 'SRP011924', 'SRP059039', 'SRP055438', 'SRP014428', 'SRP006575', 'SRP043391', 'SRP015668', 'SRP052056', 'SRP033464', 'SRP042228', 'SRP043085', 'SRP063460', 'SRP039591', 'SRP007483', 'SRP040622', 'SRP007946', 'SRP065812', 'ERP009437']


In [16]:
# Get all runs from xml
experiment_packages = root.findall('EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE')

plier_healthy = []
plier_disease = []
classifier_healthy = []
classifier_disease = []

recount_study_set = set(study_accessions)

# Find the run accessions and sort them into groups
for experiment_package in experiment_packages:
    study_accession = experiment_package.find('STUDY').get('accession')
    in_plier_set = study_accession in plier_sample_accessions
    
    case_control_status = experiment_package.get('category')
    
    if case_control_status == 'invalid' or study_accession not in recount_study_set:
        continue
    
    
    runs = experiment_package.findall('RUN_SET/RUN/IDENTIFIERS/PRIMARY_ID')
    for run in runs:
        run_id = '.'.join([study_accession, run.text])
        
        if case_control_status == 'control':
            if in_plier_set:
                plier_healthy.append(run_id)
            else:
                classifier_healthy.append(run_id)
        elif case_control_status == 'case':
            if in_plier_set:
                plier_disease.append(run_id)
            else:
                classifier_disease.append(run_id)
      

In [17]:
print(len(plier_healthy))
print(len(plier_disease))
    
print(len(classifier_healthy))
print(len(classifier_disease))

print(plier_healthy[:5])

333
887
3314
3965
['SRP061240.SRR2105249', 'SRP061240.SRR2105250', 'SRP061240.SRR2105243', 'SRP061240.SRR2105244', 'SRP061240.SRR2105221']


## Downloading Data
---
We'll now switch to R, because the only programmatic way to access recount2 data is through the recount bioconductor package. Thanks to the magic of rpy, we can use the list of run accessions we generated in the R code below.

The R portion of this notebook is based heavily on Qiewen Hu's script [here](https://github.com/greenelab/rheum-plier-data/blob/master/recount2/1-get_all_recount_dataset.R)

In [18]:
%%R 
`%>%` <- dplyr::`%>%`
library(recount)
library(xml2)

In [19]:
%%R
# Get RPKM value for each gene - adapted from recount package
getRPKM <- function(rse, length_var = 'bp_length', mapped_var = NULL) { 
  # Computes the RPKM value for each gene in the sample.
  #
  # Args: 
  #  rse: A RangedSummarizedExperiment-class object in recount package
  #  length_var: A length 1 character vector with the column name from rowData(rse) that has
  #              the coding length. For gene level objects from recount this is bp_length. If
  #              NULL, then it will use width(rowRanges(rse)) which should be used for exon RSEs.
  #  mapped_var: A length 1 character vector with the column name from colData(rse) that has
  #              the number of reads mapped. If NULL (default) then it will use the column 
  #              sums of the counts matrix
  # Returns:
  #   RPKM value for each sample
  if(!is.null(mapped_var)){
    mapped <- colData(rse)[, mapped_var] 
  } else {
    mapped <- colSums(assays(rse)$counts) 
  } 
  bg <- matrix(mapped, ncol = ncol(rse), nrow = nrow(rse), byrow = TRUE) 
  if(!is.null(length_var)){
    len <- rowData(rse)[, length_var] 
  } else {
    len <- width(rowRanges(rse))
  }
  wid <- matrix(len, nrow = nrow(rse), ncol = ncol(rse), byrow = FALSE) 
  rpkm <- assays(rse)$counts / (wid/1000) / (bg/1e6) 
  return(rpkm)
} 

getExperimentDf <- function(rpkm.list, id.vector) {
  # Extracts the runs found in id.vector from a list of dataframes, and combines
  # them into a single large dataframe
    
  rpkm.df <- do.call(base::cbind, c(rpkm.list, by = "id"))
  subset.df <- rpkm.df %>% dplyr::select(dplyr::one_of(id.vector))
  subset.df <- tibble::rownames_to_column(subset.df, "ENSG")
  
  return(subset.df)
}

In [20]:
%%R

data.dir <- file.path("../data")
recount.dir <- file.path(data.dir, "recount")
dir.create(recount.dir, recursive = TRUE, showWarnings = FALSE)

In [21]:
%%R -i study_accessions

included.study.list <- study_accessions

# Download all recount2 samples in included.study.list that are available
# This takes a while
lapply(included.study.list, 
       function(x) tryCatch(download_study(x, type = "rse-gene", 
                                           outdir = file.path(recount.dir, x)),
                            error= function(e) NULL))

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:57:06 downloading file rse_gene.Rdata to ../data/recount/SRP034698

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP034698/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/octet-stream'
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2260554 bytes (2.2 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/pytho

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 1762440 bytes (1.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 1.7 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:57:26 downloading file rse_gene.Rdata to ../data/recount/SRP058740

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP058740/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2943831 bytes (2.8 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 50.1 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:57:47 downloading file rse_gene.Rdata to ../data/recount/SRP014020

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP014020/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2338494 bytes (2.2 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:57:48 downloading file rse_gene.Rdata to ../data/recount/SRP002274

  warnings.warn(

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 6.3 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:58:02 downloading file rse_gene.Rdata to ../data/recount/SRP057586

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP057586/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 3340251 bytes (3.2 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:58:03 downloading file rse_gene.Rdata to ../data/recount/SRP056477

  warnings.warn(x

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 5.3 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:58:33 downloading file rse_gene.Rdata to ../data/recount/SRP017959

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP017959/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 3113317 bytes (3.0 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 13:58:35 downloading file rse_gene.Rdata to ../data/recount/SRP007483

  warnings.warn(x

...

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:03:32 downloading file rse_gene.Rdata to ../data/recount/SRP029452

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP029452/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2831554 bytes (2.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:03:35 downloading file rse_gene.Rdata to ../data/recount/ERP012633

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/re

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 5.4 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:03:53 downloading file rse_gene.Rdata to ../data/recount/SRP006575

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP006575/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 11585487 bytes (11.0 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 11.0 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/pyth

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:04:16 downloading file rse_gene.Rdata to ../data/recount/SRP047192

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP047192/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 9729079 bytes (9.3 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:04:18 downloading file rse_gene.Rdata to ../data/recount/ERP003613

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/re

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:04:36 downloading file rse_gene.Rdata to ../data/recount/SRP035864

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP035864/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2551004 bytes (2.4 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-20 14:04:37 downloading file rse_gene.Rdata to ../data/recount/SRP042184

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/re

[[1]]
[1] "http://duffel.rail.bio/recount/v2/SRP034698/rse_gene.Rdata"

[[2]]
[1] "http://duffel.rail.bio/recount/v2/ERP010930/rse_gene.Rdata"

[[3]]
[1] "http://duffel.rail.bio/recount/v2/SRP031459/rse_gene.Rdata"

[[4]]
[1] "http://duffel.rail.bio/recount/v2/SRP001558/rse_gene.Rdata"

[[5]]
[1] "http://duffel.rail.bio/recount/v2/SRP063838/rse_gene.Rdata"

[[6]]
[1] "http://duffel.rail.bio/recount/v2/SRP042620/rse_gene.Rdata"

[[7]]
[1] "http://duffel.rail.bio/recount/v2/SRP002326/rse_gene.Rdata"

[[8]]
[1] "http://duffel.rail.bio/recount/v2/SRP045573/rse_gene.Rdata"

[[9]]
[1] "http://duffel.rail.bio/recount/v2/SRP028738/rse_gene.Rdata"

[[10]]
[1] "http://duffel.rail.bio/recount/v2/SRP045500/rse_gene.Rdata"

[[11]]
[1] "http://duffel.rail.bio/recount/v2/ERP002414/rse_gene.Rdata"

[[12]]
[1] "http://duffel.rail.bio/recount/v2/SRP058740/rse_gene.Rdata"

[[13]]
[1] "http://duffel.rail.bio/recount/v2/SRP052896/rse_gene.Rdata"

[[14]]
[1] "http://duffel.rail.bio/recount/v2/SRP055749/rse_

In [22]:
%%R

# get RPKM for each experiment and add to list
rpkm.list <- list()
for(experiment in included.study.list) {
  possibleError <- tryCatch(load(file.path(recount.dir, experiment, 'rse_gene.Rdata')), error=function(e) e)
  # If the file can't be loaded, it wasn't in recount, so we don't want to try to add it to our
  # gene expression matrix
  if (inherits(possibleError, 'error')){
    next
  }
  
  rpkm <- as.data.frame(getRPKM(rse_gene))
  rpkm$id <- rownames(rpkm)
  rpkm.list[[experiment]] <- rpkm
}

In [23]:
%%R -i plier_healthy,plier_disease,classifier_healthy,classifier_disease


# Convert our lists of strings to vectors of strings, and change their names to match
# R naming conventions
plier.healthy <- unlist(plier_healthy)
plier.disease <- unlist(plier_disease)
classifier.healthy <- unlist(classifier_healthy)
classifier.disease <- unlist(classifier_disease)

plier.healthy.df <- getExperimentDf(rpkm.list, plier.healthy)
plier.disease.df <- getExperimentDf(rpkm.list, plier.disease)
classifier.healthy.df <- getExperimentDf(rpkm.list, classifier.healthy)
classifier.disease.df <- getExperimentDf(rpkm.list, classifier.disease)

print(dim(plier.healthy.df))
print(dim(plier.disease.df))
print(dim(classifier.healthy.df))
print(dim(classifier.disease.df))

[1] 58037   326
[1] 58037   875
[1] 58037  3182
[1] 58037  2350


In [24]:
%%R

write.table(plier.healthy.df, file.path(data.dir, 'plier_healthy.tsv'), sep='\t', row.names=FALSE)
write.table(plier.disease.df, file.path(data.dir, 'plier_disease.tsv'), sep='\t', row.names=FALSE)
write.table(classifier.healthy.df, file.path(data.dir, 'classifier_healthy.tsv'), sep='\t', row.names=FALSE)
write.table(classifier.disease.df, file.path(data.dir, 'classifier_disease.tsv'), sep='\t', row.names=FALSE)